In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
def craigslist_motorcycle_scraper(city_list):

    title = []
    price = []
    neighborhood = []
    attributes = []
    description = []
    city_names = []

    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            title_ = []
            price_ = []
            neighborhood_ = []
            attributes_ = []
            description_ = []
            city_names_ = []
            #get request      
            response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, response.status_code))

            #define the html text
            post_soup = BeautifulSoup(response.text, 'html.parser')

            #define the posts
    #         posts = page_html.find_all('li', class_= 'result-row')

            #extract data by item
    #         for post in posts:
            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                sub_post = requests.get(link)
                sub_soup = BeautifulSoup(sub_post.text, 'html.parser')

    #             if tag.find('small') is not None:

                post_title = sub_soup.find('span', id = 'titletextonly')
                title_.append(post_title)

                post_price = sub_soup.find('span', class_ = 'price')
                price_.append(post_price)

                post_neighborhood = sub_soup.find('small')
                neighborhood_.append(post_neighborhood)

                post_attributes = sub_soup.find_all('p', attrs = {'class': 'attrgroup'})
                attributes_.append(post_attributes)

                post_description = sub_soup.find('section', id = 'postingbody')
                description_.append(post_description)

                city_names_.append(city)

                time.sleep(random.randint(2,3))
                count += 1
                
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            title.append(title_)
            price.append(price_)
            neighborhood.append(neighborhood_)
            attributes.append(attributes_)
            description.append(description_)
            city_names.append(city_names_)


        print("\n")
        print("{} complete!".format(city.title()))
        print(str(len(title_)) + " rows collected.")
        print("\n")
    return title, price, neighborhood, attributes, description, city_names

In [ ]:
title = []

for _ in title15:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)

price = []

for _ in price15:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

neighborhood = []

for _ in neighborhood15:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)

attributes = []

for _ in attributes15:
    for __ in _:
        attributes.append(__)

description = []

for _ in description15:
    for __ in _:
        description.append(__)

city = []

for _ in city15:
    for __ in _:
        city.append(__)